In [ ]:
!pip install tensorflow_hub

In [ ]:
!pip install tensorflow_text

     |████████████████████████████████| 4.3 MB 7.4 MB/s 


In [21]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd

In [28]:
import chardet
with open('/content/drive/MyDrive/Colab Notebooks/TUKL summer internship/week4/spam.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result


{'confidence': 0.7270322499829184, 'encoding': 'Windows-1252', 'language': ''}

In [30]:
df= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TUKL summer internship/week4/spam.csv',encoding='Windows-1252')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [33]:
df=df[['v1','v2']]
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [34]:
df= df.rename(columns={'v1': 'Category'})

In [35]:
df=df.rename(columns={'v2': 'Message'})

In [36]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [37]:
df.groupby('Category').describe()

Message                                                               
           count unique                                                top freq
Category                                                                       
ham         4825   4516                             Sorry, I'll call later   30
spam         747    653  Please call our customer service representativ...    4

In [38]:
df['spam']=df['Category'].apply(lambda x: 1 if x=='spam' else 0)
df.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [39]:
from sklearn.model_selection import train_test_split#stratify ensures balance between spam and no spam in test and train

X_train, X_test, y_train, y_test = train_test_split(df['Message'],df['spam'], stratify=df['spam'])

In [41]:
y_test.value_counts()

0    1206
1     187
Name: spam, dtype: int64

In [42]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [43]:
def get_sentence_embeding(sentences):#generates embedding vector for sentence
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [44]:
get_sentence_embeding([
    "500$ discount. hurry up", 
    "Bhavin, are you up for a volleybal game tomorrow?"]
)

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.84351707, -0.51327264, -0.888457  , ..., -0.74748856,
        -0.75314736,  0.91964495],
       [-0.87208384, -0.5054399 , -0.9444669 , ..., -0.8584754 ,
        -0.7174538 ,  0.88082993]], dtype=float32)>

In [45]:
e = get_sentence_embeding([
    "banana", #using this to generate similarity
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
]
)

In [46]:

from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[0]],[e[1]])

array([[0.9911088]], dtype=float32)

In [47]:
cosine_similarity([e[0]],[e[3]])

array([[0.84703803]], dtype=float32)

In [48]:
cosine_similarity([e[3]],[e[4]])

array([[0.9872034]], dtype=float32)

In [49]:
#so far we have used sequential models but now we are using BERT which is a functional model.
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')#name can be anything #input shape variable #data type string
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [50]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer_2 (KerasLayer)      {'input_mask': (None 0           text[0][0]                       
__________________________________________________________________________________________________
keras_layer_3 (KerasLayer)      {'encoder_outputs':  109482241   keras_layer_2[0][0]              
                                                                 keras_layer_2[0][1]              
                                                                 keras_layer_2[0][2]              
______________________________________________________________________________________________

In [51]:
len(X_train)

4179

In [52]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [53]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
131/131 [==============================] - 1244s 9s/step - loss: 0.3931 - accuracy: 0.8418
Epoch 2/5
131/131 [==============================] - 1236s 9s/step - loss: 0.2655 - accuracy: 0.8784
Epoch 3/5
131/131 [==============================] - 1241s 9s/step - loss: 0.2193 - accuracy: 0.9074
Epoch 4/5
131/131 [==============================] - 1245s 10s/step - loss: 0.1896 - accuracy: 0.9256
Epoch 5/5
131/131 [==============================] - 1239s 9s/step - loss: 0.1733 - accuracy: 0.9337


In [54]:
model.evaluate(X_test, y_test)

44/44 [==============================] - 413s 9s/step - loss: 0.1671 - accuracy: 0.9232


[0.1671140193939209, 0.9231873750686646]